## 1

In [27]:
import os
import json
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
import glob

class CustomSegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_paths = glob.glob(os.path.join(image_dir, '*.jpg'))
        self.mask_paths = [os.path.join(mask_dir, f"Mask_{os.path.basename(p).replace('.jpg', '.png')}") for p in self.image_paths]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # 이미지 및 마스크 경로 가져오기
        image_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]
        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path)

        # 이미지와 마스크에 동일한 변환 적용
        if self.transform:
            image = self.transform(image)
            mask = mask.resize((256, 256), resample=Image.NEAREST)
        
        # 마스크는 PIL 이미지를 NumPy로 변환 후 텐서로 변환
        mask_np = np.array(mask, dtype=np.int64)
        mask_tensor = torch.from_numpy(mask_np).long()

        return image, mask_tensor

# 변환 설정 (리사이즈 포함)
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 이미지 크기 조정
    transforms.ToTensor(),  # 텐서로 변환
])

# 원본 이미지와 마스크 이미지 경로 설정
train_image_dirs = ['./Surface_1', './Surface_2', './Surface_3', './Surface_4']  # 학습용 디렉토리
val_image_dir = './Surface_5'  # 검증용 디렉토리
train_mask_dirs = ['./Surface_1_mask', './Surface_2_mask', './Surface_3_mask', './Surface_4_mask']
val_mask_dir = './Surface_5_mask'

# 각 학습 디렉토리에 대해 데이터로더 생성
train_dataloaders = [DataLoader(CustomSegmentationDataset(image_dir, mask_dir, transform=transform), batch_size=64, shuffle=True) 
                     for image_dir, mask_dir in zip(train_image_dirs, train_mask_dirs)]

# 검증 데이터셋 및 데이터로더 준비
val_dataset = CustomSegmentationDataset(val_image_dir, val_mask_dir, transform=transform)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

print("Train and validation dataloaders are ready!")

# 데이터 로더에서 샘플 확인
for i, dataloader in enumerate(train_dataloaders):
    try:
        images, masks = next(iter(dataloader))
        print(f"[Train Dataloader {i + 1}] 샘플 배치 로드 성공!")
        print("이미지 텐서 크기:", images.size())
        print("마스크 텐서 크기:", masks.size())
        print(f"[Train Dataloader {i + 1}] 마스크의 고유 클래스 ID:", torch.unique(masks))
    except Exception as e:
        print(f"Train Dataloader {i + 1} 오류 발생:", e)

try:
    images, masks = next(iter(val_dataloader))
    print("[Validation Dataloader] 샘플 배치 로드 성공!")
    print("이미지 텐서 크기:", images.size())
    print("마스크 텐서 크기:", masks.size())
    print("[Validation Dataloader] 마스크의 고유 클래스 ID:", torch.unique(masks))
except Exception as e:
    print("Validation 데이터 로더 오류 발생:", e)


Train and validation dataloaders are ready!
[Train Dataloader 1] 샘플 배치 로드 성공!
이미지 텐서 크기: torch.Size([64, 3, 256, 256])
마스크 텐서 크기: torch.Size([64, 256, 256])
[Train Dataloader 1] 마스크의 고유 클래스 ID: tensor([0, 1, 2, 3, 4, 5, 6, 7])
[Train Dataloader 2] 샘플 배치 로드 성공!
이미지 텐서 크기: torch.Size([64, 3, 256, 256])
마스크 텐서 크기: torch.Size([64, 256, 256])
[Train Dataloader 2] 마스크의 고유 클래스 ID: tensor([0, 1, 2, 3, 4, 5, 6, 7])
[Train Dataloader 3] 샘플 배치 로드 성공!
이미지 텐서 크기: torch.Size([64, 3, 256, 256])
마스크 텐서 크기: torch.Size([64, 256, 256])
[Train Dataloader 3] 마스크의 고유 클래스 ID: tensor([0, 1, 2, 3, 4, 5, 6, 7])
[Train Dataloader 4] 샘플 배치 로드 성공!
이미지 텐서 크기: torch.Size([64, 3, 256, 256])
마스크 텐서 크기: torch.Size([64, 256, 256])
[Train Dataloader 4] 마스크의 고유 클래스 ID: tensor([0, 1, 2, 3, 4, 5, 6, 7])
[Validation Dataloader] 샘플 배치 로드 성공!
이미지 텐서 크기: torch.Size([64, 3, 256, 256])
마스크 텐서 크기: torch.Size([64, 256, 256])
[Validation Dataloader] 마스크의 고유 클래스 ID: tensor([0, 1, 2, 3, 4, 5, 6, 7])


In [28]:
# 학습 데이터셋의 이미지 개수 확인
total_train_samples = sum(len(dataloader.dataset) for dataloader in train_dataloaders)
print("Train dataset 이미지 개수:", total_train_samples)

# 검증 데이터셋의 이미지 개수 확인
print("Validation dataset 이미지 개수:", len(val_dataset))


Train dataset 이미지 개수: 37607
Validation dataset 이미지 개수: 8792


In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

# 경량화된 ASPP 모듈
class LightweightASPP(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(LightweightASPP, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=3, dilation=3)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        self.bn3 = nn.BatchNorm2d(out_channels)

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.conv_pool = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.bn_pool = nn.BatchNorm2d(out_channels)

        self.conv_out = nn.Conv2d(out_channels * 4, out_channels, kernel_size=1)

    def forward(self, x):
        x1 = F.relu(self.bn1(self.conv1(x)))
        x2 = F.relu(self.bn2(self.conv2(x)))
        x3 = F.relu(self.bn3(self.conv3(x)))

        x4 = self.pool(x)
        x4 = F.relu(self.bn_pool(self.conv_pool(x4)))
        x4 = F.interpolate(x4, size=x3.size()[2:], mode='bilinear', align_corners=False)

        x = torch.cat((x1, x2, x3, x4), dim=1)
        x = self.conv_out(x)
        return x

# 경량화된 DeepLabV3 모델
class LightweightDeepLabV3(nn.Module):
    def __init__(self, num_classes):
        super(LightweightDeepLabV3, self).__init__()

        # MobileNetV2 백본 사용
        backbone = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
        self.backbone = nn.Sequential(
            backbone.features,
            nn.Conv2d(1280, 320, kernel_size=1)  # 마지막 레이어를 압축
        )

        # 경량화된 ASPP 모듈
        self.aspp = LightweightASPP(in_channels=320, out_channels=128)

        # 피처 조정을 위한 추가 Conv 레이어
        self.conv1 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(128)

        # 최종 출력 클래스 수에 맞추기
        self.conv2 = nn.Conv2d(128, num_classes, kernel_size=1)

    def forward(self, x):
        x_size = x.size()[2:]

        x = self.backbone(x)

        x = self.aspp(x)

        x = F.relu(self.bn1(self.conv1(x)))
        x = self.conv2(x)

        x = F.interpolate(x, size=x_size, mode='bilinear', align_corners=False)
        return x

# GPU 또는 CPU에 모델 로드
model = LightweightDeepLabV3(num_classes=8)  # 클래스 수에 맞게 설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


In [30]:
import torch
import os
import csv
import numpy as np
from tqdm import tqdm

# Class names definition
CLASS_NAMES = ["Background", "Sidewalk", "Braille Guide Blocks", "Roadway", "Alley", "Bike Lane", "Caution Zone", "Cross walk"]

# 성능 지표를 CSV 파일에 저장하는 함수 (클래스별 mIoU 추가, 배경 클래스 제외)
def save_metrics_to_csv(epoch, train_loss, val_loss, train_miou, val_miou, class_train_iou, class_val_iou, filename="resize_metrics.csv"):
    try:
        file_exists = os.path.isfile(filename)
        
        with open(filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            if not file_exists:
                # CSV 헤더에 클래스별 mIoU 추가 (배경 제외)
                header = ["Epoch", "Train Loss", "Val Loss", "Train mIoU", "Val mIoU"] + \
                         [f"Train IoU - {name}" for name in CLASS_NAMES[1:]] + \
                         [f"Val IoU - {name}" for name in CLASS_NAMES[1:]]
                writer.writerow(header)
            
            # 배경 클래스를 제외한 클래스별 mIoU와 함께 데이터를 작성
            row = [epoch, train_loss, val_loss, train_miou, val_miou] + \
                  class_train_iou + class_val_iou
            writer.writerow(row)
        print(f"Metrics saved successfully to {filename}.")
    except Exception as e:
        print(f"Failed to save metrics to CSV: {e}")

# Function to save model checkpoints
def save_checkpoint(epoch, model, optimizer, checkpoint_dir, filename="last_checkpoint.pth"):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    checkpoint_filename = os.path.join(checkpoint_dir, filename)
    torch.save(checkpoint, checkpoint_filename)
    print(f"Checkpoint saved at epoch {epoch} as {checkpoint_filename}.")

# Function to load model checkpoints
def load_checkpoint(filename, model, optimizer=None):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    print(f"Checkpoint loaded from epoch {epoch}.")
    return epoch

# Function to calculate IoU per class
def calculate_iou_per_class(pred, target, num_classes):
    iou_per_class = []
    for cls in range(num_classes):
        pred_inds = pred == cls
        target_inds = target == cls
        intersection = (pred_inds & target_inds).sum().float().item()
        union = (pred_inds | target_inds).sum().float().item()
        if union == 0:
            iou_per_class.append(float('nan'))
        else:
            iou_per_class.append(intersection / union)
    return iou_per_class

# 모델 훈련 함수 (CSV에 클래스별 mIoU 저장 추가)
def train_model_with_csv_and_checkpoint(model, train_dataloader, val_dataloader, criterion, optimizer, num_classes, num_epochs=25, device='cuda:0', csv_filename="resize_metrics.csv", checkpoint_dir="./resize_checkpoints", checkpoint_file=None):
    os.makedirs(checkpoint_dir, exist_ok=True)
    model.to(device)
    print("모델 장치:", next(model.parameters()).device)

    best_val_miou = -1
    start_epoch = 0
    if checkpoint_file and os.path.isfile(checkpoint_file):
        start_epoch = load_checkpoint(checkpoint_file, model, optimizer)
        print(f"체크포인트 로드, {start_epoch + 1} 에폭부터 재개합니다.")
    else:
        print("체크포인트가 없거나 파일을 찾을 수 없습니다. 처음부터 학습을 시작합니다.")

    for epoch in range(start_epoch, start_epoch + num_epochs):
        # 현재 에폭에 사용할 DataLoader 선택
        dataloader_index = epoch % len(train_dataloader)  # 순환하면서 선택
        current_dataloader = train_dataloader[dataloader_index]

        print(f"{epoch + 1}/{start_epoch + num_epochs} epoch start with Train Dataloader {dataloader_index + 1}")

        model.train()
        running_loss = 0.0
        train_iou = [0.0] * num_classes
        total_train_samples = 0

        # 선택된 DataLoader만 사용하여 학습
        for images, masks in tqdm(current_dataloader, desc=f"Train Epoch {epoch + 1}, Dataloader {dataloader_index + 1}", leave=False):
            images = images.to(device)
            masks = masks.long().to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()

            outputs = torch.argmax(outputs, dim=1)
            running_loss += loss.item() * images.size(0)
            total_train_samples += images.size(0)

            # 클래스별 IoU 계산
            class_iou = calculate_iou_per_class(outputs, masks, num_classes)
            for j in range(num_classes):
                if not np.isnan(class_iou[j]):
                    train_iou[j] += class_iou[j]

        # 평균 IoU 및 손실 계산
        epoch_loss = running_loss / total_train_samples
        train_iou = [x / len(current_dataloader) for x in train_iou]
        train_miou = np.nanmean(train_iou[1:])  # 배경 클래스 제외

        print(f"{epoch + 1} epoch - Train Loss: {epoch_loss:.4f}, Train mIoU: {train_miou:.4f}")
        print("classes Train IoU:", [f"{CLASS_NAMES[i]}: {iou:.4f}" for i, iou in enumerate(train_iou[1:], start=1)])

        # 검증 단계는 매 에폭마다 수행
        model.eval()
        val_running_loss = 0.0
        val_iou = [0.0] * num_classes

        with torch.no_grad():
            for images, masks in val_dataloader:
                images = images.to(device)
                masks = masks.long().to(device)

                outputs = model(images)
                loss = criterion(outputs, masks)
                outputs = torch.argmax(outputs, dim=1)
                val_running_loss += loss.item() * images.size(0)

                # 클래스별 IoU 계산
                class_iou = calculate_iou_per_class(outputs, masks, num_classes)
                for j in range(num_classes):
                    if not np.isnan(class_iou[j]):
                        val_iou[j] += class_iou[j]

        # 검증 데이터에 대한 평균 IoU 및 손실 계산
        val_loss = val_running_loss / len(val_dataloader.dataset)
        val_iou = [x / len(val_dataloader) for x in val_iou]
        val_miou = np.nanmean(val_iou[1:])  # 배경 클래스 제외

        print(f"{epoch + 1} epoch - Validation Loss: {val_loss:.4f}, Validation mIoU: {val_miou:.4f}")
        print("classes Validation IoU:", [f"{CLASS_NAMES[i]}: {iou:.4f}" for i, iou in enumerate(val_iou[1:], start=1)])

        # 성능 지표를 CSV에 저장
        save_metrics_to_csv(epoch + 1, epoch_loss, val_loss, train_miou, val_miou, train_iou[1:], val_iou[1:])

        # 최적의 검증 성능 기준으로 체크포인트 저장
        if val_miou > best_val_miou:
            best_val_miou = val_miou
            best_checkpoint_filename = f"best_checkpoint_epoch_{epoch + 1}.pth"
            save_checkpoint(epoch + 1, model, optimizer, checkpoint_dir, filename=best_checkpoint_filename)

    # 마지막 에폭 체크포인트 저장
    last_checkpoint_filename = f"last_checkpoint_epoch_{epoch + 1}.pth"
    save_checkpoint(epoch + 1, model, optimizer, checkpoint_dir, filename=last_checkpoint_filename)
    print("훈련 완료.")


In [31]:
import torch.optim as optim

# CUDA 환경 설정
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

# 모델 및 학습 설정
num_classes = 8  # 실제 학습할 클래스 수 (배경 제외)
learning_rate = 0.0007  # 논문에 나와있는 learning rate
weight_decay = 0.0005
num_epochs = 100


# 모델, 옵티마이저, 손실 함수 설정
model = LightweightDeepLabV3(num_classes=num_classes)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# 손실 함수 및 장치 설정 (가중치 없이)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
criterion = torch.nn.CrossEntropyLoss(ignore_index=0).to(device)

# 체크포인트 파일 지정 
checkpoint_file = "./"

# 학습 함수 호출
train_model_with_csv_and_checkpoint(
    model=model,
    train_dataloader=train_dataloaders,
    val_dataloader=val_dataloader,
    criterion=criterion,
    optimizer=optimizer,
    num_classes=num_classes,
    num_epochs=num_epochs,
    device=device,
    csv_filename="./resize_metrics.csv",
    checkpoint_dir="./resize_checkpoints",
    checkpoint_file=checkpoint_file  # 여기에 원하는 체크포인트 파일을 지정
)


모델 장치: cuda:0
체크포인트가 없거나 파일을 찾을 수 없습니다. 처음부터 학습을 시작합니다.
1/100 epoch start with Train Dataloader 1


1 epoch - Train Loss: 0.5718, Train mIoU: 0.3691
classes Train IoU: ['Sidewalk: 0.6828', 'Braille Guide Blocks: 0.1533', 'Roadway: 0.2810', 'Alley: 0.6652', 'Bike Lane: 0.5045', 'Caution Zone: 0.1937', 'Cross walk: 0.1033']
1 epoch - Validation Loss: 0.4565, Validation mIoU: 0.4461
classes Validation IoU: ['Sidewalk: 0.6834', 'Braille Guide Blocks: 0.4840', 'Roadway: 0.2566', 'Alley: 0.7144', 'Bike Lane: 0.5048', 'Caution Zone: 0.2862', 'Cross walk: 0.1935']
Metrics saved successfully to resize_metrics.csv.
Checkpoint saved at epoch 1 as ./resize_checkpoints/best_checkpoint_epoch_1.pth.
2/100 epoch start with Train Dataloader 2


2 epoch - Train Loss: 0.3786, Train mIoU: 0.4624
classes Train IoU: ['Sidewalk: 0.6994', 'Braille Guide Blocks: 0.4272', 'Roadway: 0.3528', 'Alley: 0.7279', 'Bike Lane: 0.5719', 'Caution Zone: 0.2824', 'Cross walk: 0.1750']
2 epoch - Validation Loss: 0.3881, Validation mIoU: 0.4858
classes Validation IoU: ['Sidewalk: 0.6586', 'Braille Guide Blocks: 0.5276', 'Roadway: 0.2914', 'Alley: 0.7411', 'Bike Lane: 0.5820', 'Caution Zone: 0.3308', 'Cross walk: 0.2690']
Metrics saved successfully to resize_metrics.csv.
Checkpoint saved at epoch 2 as ./resize_checkpoints/best_checkpoint_epoch_2.pth.
3/100 epoch start with Train Dataloader 3


3 epoch - Train Loss: 0.3787, Train mIoU: 0.4775
classes Train IoU: ['Sidewalk: 0.6771', 'Braille Guide Blocks: 0.4856', 'Roadway: 0.3689', 'Alley: 0.7135', 'Bike Lane: 0.6075', 'Caution Zone: 0.3013', 'Cross walk: 0.1885']
3 epoch - Validation Loss: 0.3696, Validation mIoU: 0.5105
classes Validation IoU: ['Sidewalk: 0.6692', 'Braille Guide Blocks: 0.5881', 'Roadway: 0.3750', 'Alley: 0.7515', 'Bike Lane: 0.6067', 'Caution Zone: 0.3613', 'Cross walk: 0.2220']
Metrics saved successfully to resize_metrics.csv.
Checkpoint saved at epoch 3 as ./resize_checkpoints/best_checkpoint_epoch_3.pth.
4/100 epoch start with Train Dataloader 4


4 epoch - Train Loss: 0.3644, Train mIoU: 0.5015
classes Train IoU: ['Sidewalk: 0.6826', 'Braille Guide Blocks: 0.5605', 'Roadway: 0.3808', 'Alley: 0.7418', 'Bike Lane: 0.5830', 'Caution Zone: 0.3389', 'Cross walk: 0.2227']
4 epoch - Validation Loss: 0.3232, Validation mIoU: 0.5324
classes Validation IoU: ['Sidewalk: 0.6864', 'Braille Guide Blocks: 0.6029', 'Roadway: 0.3266', 'Alley: 0.7521', 'Bike Lane: 0.6270', 'Caution Zone: 0.4469', 'Cross walk: 0.2848']
Metrics saved successfully to resize_metrics.csv.
Checkpoint saved at epoch 4 as ./resize_checkpoints/best_checkpoint_epoch_4.pth.
5/100 epoch start with Train Dataloader 1


...

45 epoch - Train Loss: 0.2861, Train mIoU: 0.5514
classes Train IoU: ['Sidewalk: 0.6851', 'Braille Guide Blocks: 0.5665', 'Roadway: 0.4902', 'Alley: 0.7550', 'Bike Lane: 0.6872', 'Caution Zone: 0.4470', 'Cross walk: 0.2287']
45 epoch - Validation Loss: 0.3662, Validation mIoU: 0.5162
classes Validation IoU: ['Sidewalk: 0.6761', 'Braille Guide Blocks: 0.6090', 'Roadway: 0.3397', 'Alley: 0.7469', 'Bike Lane: 0.6539', 'Caution Zone: 0.3070', 'Cross walk: 0.2807']
Metrics saved successfully to resize_metrics.csv.
46/100 epoch start with Train Dataloader 2


46 epoch - Train Loss: 0.2678, Train mIoU: 0.5544
classes Train IoU: ['Sidewalk: 0.6946', 'Braille Guide Blocks: 0.5764', 'Roadway: 0.4644', 'Alley: 0.7701', 'Bike Lane: 0.6665', 'Caution Zone: 0.4565', 'Cross walk: 0.2521']
46 epoch - Validation Loss: 0.3334, Validation mIoU: 0.5489
classes Validation IoU: ['Sidewalk: 0.6765', 'Braille Guide Blocks: 0.6638', 'Roadway: 0.3842', 'Alley: 0.7416', 'Bike Lane: 0.6378', 'Caution Zone: 0.3936', 'Cross walk: 0.3448']
Metrics saved successfully to resize_metrics.csv.
47/100 epoch start with Train Dataloader 3


47 epoch - Train Loss: 0.3063, Train mIoU: 0.5380
classes Train IoU: ['Sidewalk: 0.6764', 'Braille Guide Blocks: 0.5839', 'Roadway: 0.4430', 'Alley: 0.7427', 'Bike Lane: 0.6602', 'Caution Zone: 0.4063', 'Cross walk: 0.2538']
47 epoch - Validation Loss: 0.3448, Validation mIoU: 0.5116
classes Validation IoU: ['Sidewalk: 0.6696', 'Braille Guide Blocks: 0.6248', 'Roadway: 0.3812', 'Alley: 0.7674', 'Bike Lane: 0.6108', 'Caution Zone: 0.2592', 'Cross walk: 0.2682']
Metrics saved successfully to resize_metrics.csv.
48/100 epoch start with Train Dataloader 4


48 epoch - Train Loss: 0.2961, Train mIoU: 0.5494
classes Train IoU: ['Sidewalk: 0.6900', 'Braille Guide Blocks: 0.6104', 'Roadway: 0.4366', 'Alley: 0.7680', 'Bike Lane: 0.6453', 'Caution Zone: 0.4336', 'Cross walk: 0.2620']
48 epoch - Validation Loss: 0.3021, Validation mIoU: 0.5493
classes Validation IoU: ['Sidewalk: 0.6889', 'Braille Guide Blocks: 0.6620', 'Roadway: 0.3699', 'Alley: 0.7668', 'Bike Lane: 0.6463', 'Caution Zone: 0.4186', 'Cross walk: 0.2927']
Metrics saved successfully to resize_metrics.csv.
49/100 epoch start with Train Dataloader 1


KeyboardInterrupt: 

## 2

In [ ]:
import torch
import os
import csv
import numpy as np
from tqdm import tqdm

# Class names definition
CLASS_NAMES = ["Background", "Sidewalk", "Braille Guide Blocks", "Roadway", "Alley", "Bike Lane", "Caution Zone", "Cross walk"]

# 성능 지표를 CSV 파일에 저장하는 함수 (클래스별 mIoU 추가, 배경 클래스 제외)
def save_metrics_to_csv(epoch, train_loss, val_loss, train_miou, val_miou, class_train_iou, class_val_iou, filename="resize_metrics.csv"):
    try:
        file_exists = os.path.isfile(filename)
        
        with open(filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            if not file_exists:
                # CSV 헤더에 클래스별 mIoU 추가 (배경 제외)
                header = ["Epoch", "Train Loss", "Val Loss", "Train mIoU", "Val mIoU"] + \
                         [f"Train IoU - {name}" for name in CLASS_NAMES[1:]] + \
                         [f"Val IoU - {name}" for name in CLASS_NAMES[1:]]
                writer.writerow(header)
            
            # 배경 클래스를 제외한 클래스별 mIoU와 함께 데이터를 작성
            row = [epoch, train_loss, val_loss, train_miou, val_miou] + \
                  class_train_iou + class_val_iou
            writer.writerow(row)
        print(f"Metrics saved successfully to {filename}.")
    except Exception as e:
        print(f"Failed to save metrics to CSV: {e}")

# Function to save model checkpoints
def save_checkpoint(epoch, model, optimizer, checkpoint_dir, filename="last_checkpoint.pth"):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    checkpoint_filename = os.path.join(checkpoint_dir, filename)
    torch.save(checkpoint, checkpoint_filename)
    print(f"Checkpoint saved at epoch {epoch} as {checkpoint_filename}.")

# Function to load model checkpoints
def load_checkpoint(filename, model, optimizer=None):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    print(f"Checkpoint loaded from epoch {epoch}.")
    return epoch

# Function to calculate IoU per class
def calculate_iou_per_class(pred, target, num_classes):
    iou_per_class = []
    for cls in range(num_classes):
        pred_inds = pred == cls
        target_inds = target == cls
        intersection = (pred_inds & target_inds).sum().float().item()
        union = (pred_inds | target_inds).sum().float().item()
        if union == 0:
            iou_per_class.append(float('nan'))
        else:
            iou_per_class.append(intersection / union)
    return iou_per_class


def train_model_with_csv_and_checkpoint(model, train_dataloader, val_dataloader, criterion, optimizer, num_classes, num_epochs=25, device='cuda:0', csv_filename="resize_metrics.csv", checkpoint_dir="./resize_checkpoints", checkpoint_file=None):
    os.makedirs(checkpoint_dir, exist_ok=True)
    model.to(device)
    print("모델 장치:", next(model.parameters()).device)

    best_val_miou = -1

    # 체크포인트 파일을 불러오고, 이어서 학습할 시작 에폭 설정
    start_epoch = 0
    if checkpoint_file and os.path.isfile(checkpoint_file):
        start_epoch = load_checkpoint(checkpoint_file, model, optimizer)
        print(f"체크포인트 로드 완료. {start_epoch + 1} 에폭부터 재개합니다.")
    else:
        print("체크포인트가 없거나 파일을 찾을 수 없습니다. 처음부터 학습을 시작합니다.")

    # CSV에 기록할 에폭을 49부터 시작하도록 설정
    csv_epoch = 49

    # 학습 루프: `start_epoch + 1`부터 `start_epoch + num_epochs`까지
    for epoch in range(start_epoch + 1, start_epoch + num_epochs + 1):
        dataloader_index = epoch % len(train_dataloader)
        current_dataloader = train_dataloader[dataloader_index]
        
        print("\n=========================================================================")
        print(f"{epoch}/{start_epoch + num_epochs} epoch start with Train Dataloader {dataloader_index + 1}")

        model.train()
        running_loss = 0.0
        train_iou = [0.0] * num_classes
        total_train_samples = 0

        for images, masks in tqdm(current_dataloader, desc=f"Train Epoch {epoch}, Dataloader {dataloader_index + 1}", leave=False):
            images = images.to(device)
            masks = masks.long().to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()

            outputs = torch.argmax(outputs, dim=1)
            running_loss += loss.item() * images.size(0)
            total_train_samples += images.size(0)

            class_iou = calculate_iou_per_class(outputs, masks, num_classes)
            for j in range(num_classes):
                if not np.isnan(class_iou[j]):
                    train_iou[j] += class_iou[j]

        epoch_loss = running_loss / total_train_samples
        train_iou = [x / len(current_dataloader) for x in train_iou]
        train_miou = np.nanmean(train_iou[1:])

        print(f"{epoch} epoch - Train Loss: {epoch_loss:.4f}, Train mIoU: {train_miou:.4f}")

        # 검증 단계 수행
        model.eval()
        val_running_loss = 0.0
        val_iou = [0.0] * num_classes

        with torch.no_grad():
            for images, masks in val_dataloader:
                images = images.to(device)
                masks = masks.long().to(device)

                outputs = model(images)
                loss = criterion(outputs, masks)
                outputs = torch.argmax(outputs, dim=1)
                val_running_loss += loss.item() * images.size(0)

                class_iou = calculate_iou_per_class(outputs, masks, num_classes)
                for j in range(num_classes):
                    if not np.isnan(class_iou[j]):
                        val_iou[j] += class_iou[j]

        val_loss = val_running_loss / len(val_dataloader.dataset)
        val_iou = [x / len(val_dataloader) for x in val_iou]
        val_miou = np.nanmean(val_iou[1:])

        print(f"{epoch} epoch - Validation Loss: {val_loss:.4f}, Validation mIoU: {val_miou:.4f}")

        # 강제로 CSV에 기록할 에폭을 49부터 시작하도록 설정
        save_metrics_to_csv(csv_epoch, epoch_loss, val_loss, train_miou, val_miou, train_iou[1:], val_iou[1:])
        csv_epoch += 1  # CSV 에폭을 증가시켜 다음 에폭에 반영

        # 매 에폭마다 체크포인트 저장
        epoch_checkpoint_filename = f"checkpoint_epoch_{epoch}.pth"
        save_checkpoint(epoch, model, optimizer, checkpoint_dir, filename=epoch_checkpoint_filename)
        

        # 최적의 검증 성능 기준으로 최상의 체크포인트도 업데이트
        if val_miou > best_val_miou:
            best_val_miou = val_miou
            best_checkpoint_filename = f"best_checkpoint_epoch_{epoch}.pth"
            save_checkpoint(epoch, model, optimizer, checkpoint_dir, filename=best_checkpoint_filename)

    # 마지막 에폭 체크포인트 저장
    last_checkpoint_filename = f"last_checkpoint_epoch_{epoch}.pth"
    save_checkpoint(epoch, model, optimizer, checkpoint_dir, filename=last_checkpoint_filename)
    print("훈련 완료.")

In [39]:
import torch.optim as optim

# CUDA 환경 설정
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

# 모델 및 학습 설정
num_classes = 8  # 실제 학습할 클래스 수 (배경 제외)
learning_rate = 0.0001  # 논문에 나와있는 learning rate
weight_decay = 0.0001
num_epochs = 100


# 모델, 옵티마이저, 손실 함수 설정
model = LightweightDeepLabV3(num_classes=num_classes)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# 손실 함수 및 장치 설정 (가중치 없이)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
criterion = torch.nn.CrossEntropyLoss(ignore_index=0).to(device)

# 체크포인트 파일 지정 
checkpoint_file = "./resize_checkpoints/best_checkpoint_epoch_32.pth"

# 학습 함수 호출
train_model_with_csv_and_checkpoint(
    model=model,
    train_dataloader=train_dataloaders,
    val_dataloader=val_dataloader,
    criterion=criterion,
    optimizer=optimizer,
    num_classes=num_classes,
    num_epochs=num_epochs,
    device=device,
    csv_filename="./resize_metrics.csv",
    checkpoint_dir="./resize_checkpoints",
    checkpoint_file="./resize_checkpoints/best_checkpoint_epoch_32.pth"  # 여기에 원하는 체크포인트 파일을 지정
)


/tmp/ipykernel_2775340/92660943.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename)


모델 장치: cuda:0
Checkpoint loaded from epoch 32.
체크포인트 로드 완료. 33 에폭부터 재개합니다.
33/132 epoch start with Train Dataloader 2


33 epoch - Train Loss: 0.2803, Train mIoU: 0.5474
33 epoch - Validation Loss: 0.3404, Validation mIoU: 0.5275
Metrics saved successfully to resize_metrics.csv.
Checkpoint saved at epoch 33 as ./resize_checkpoints/checkpoint_epoch_33.pth.
Checkpoint saved at epoch 33 as ./resize_checkpoints/best_checkpoint_epoch_33.pth.
34/132 epoch start with Train Dataloader 3


34 epoch - Train Loss: 0.3082, Train mIoU: 0.5335
34 epoch - Validation Loss: 0.3278, Validation mIoU: 0.5350
Metrics saved successfully to resize_metrics.csv.
Checkpoint saved at epoch 34 as ./resize_checkpoints/checkpoint_epoch_34.pth.
Checkpoint saved at epoch 34 as ./resize_checkpoints/best_checkpoint_epoch_34.pth.
35/132 epoch start with Train Dataloader 4


35 epoch - Train Loss: 0.3038, Train mIoU: 0.5508
35 epoch - Validation Loss: 0.3076, Validation mIoU: 0.5413
Metrics saved successfully to resize_metrics.csv.
Checkpoint saved at epoch 35 as ./resize_checkpoints/checkpoint_epoch_35.pth.
Checkpoint saved at epoch 35 as ./resize_checkpoints/best_checkpoint_epoch_35.pth.
36/132 epoch start with Train Dataloader 1


...

46 epoch - Train Loss: 0.2993, Train mIoU: 0.5388
46 epoch - Validation Loss: 0.3260, Validation mIoU: 0.5320
Metrics saved successfully to resize_metrics.csv.
Checkpoint saved at epoch 46 as ./resize_checkpoints/checkpoint_epoch_46.pth.
47/132 epoch start with Train Dataloader 4


47 epoch - Train Loss: 0.2945, Train mIoU: 0.5495
47 epoch - Validation Loss: 0.3134, Validation mIoU: 0.5493
Metrics saved successfully to resize_metrics.csv.
Checkpoint saved at epoch 47 as ./resize_checkpoints/checkpoint_epoch_47.pth.
Checkpoint saved at epoch 47 as ./resize_checkpoints/best_checkpoint_epoch_47.pth.
48/132 epoch start with Train Dataloader 1


48 epoch - Train Loss: 0.2968, Train mIoU: 0.5560
48 epoch - Validation Loss: 0.3706, Validation mIoU: 0.5241
Metrics saved successfully to resize_metrics.csv.
Checkpoint saved at epoch 48 as ./resize_checkpoints/checkpoint_epoch_48.pth.
49/132 epoch start with Train Dataloader 2


49 epoch - Train Loss: 0.2649, Train mIoU: 0.5540
49 epoch - Validation Loss: 0.3158, Validation mIoU: 0.5420
Metrics saved successfully to resize_metrics.csv.
Checkpoint saved at epoch 49 as ./resize_checkpoints/checkpoint_epoch_49.pth.
50/132 epoch start with Train Dataloader 3


KeyboardInterrupt: 

## 3. 배치사이즈 64->128 , 러닝레이트 줄이기

In [40]:
train_dataloaders = [DataLoader(CustomSegmentationDataset(image_dir, mask_dir, transform=transform), batch_size=128, shuffle=True) 
                     for image_dir, mask_dir in zip(train_image_dirs, train_mask_dirs)]

val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False)

In [44]:
import torch
import os
import csv
import numpy as np
from tqdm import tqdm

# Class names definition
CLASS_NAMES = ["Background", "Sidewalk", "Braille Guide Blocks", "Roadway", "Alley", "Bike Lane", "Caution Zone", "Cross walk"]

# 성능 지표를 CSV 파일에 저장하는 함수 (클래스별 mIoU 추가, 배경 클래스 제외)
def save_metrics_to_csv(epoch, train_loss, val_loss, train_miou, val_miou, class_train_iou, class_val_iou, filename="resize_metrics.csv"):
    try:
        file_exists = os.path.isfile(filename)
        
        with open(filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            if not file_exists:
                # CSV 헤더에 클래스별 mIoU 추가 (배경 제외)
                header = ["Epoch", "Train Loss", "Val Loss", "Train mIoU", "Val mIoU"] + \
                         [f"Train IoU - {name}" for name in CLASS_NAMES[1:]] + \
                         [f"Val IoU - {name}" for name in CLASS_NAMES[1:]]
                writer.writerow(header)
            
            # 배경 클래스를 제외한 클래스별 mIoU와 함께 데이터를 작성
            row = [epoch, train_loss, val_loss, train_miou, val_miou] + \
                  class_train_iou + class_val_iou
            writer.writerow(row)
        print(f"Metrics saved successfully to {filename}.")
    except Exception as e:
        print(f"Failed to save metrics to CSV: {e}")

# Function to save model checkpoints
def save_checkpoint(epoch, model, optimizer, checkpoint_dir, filename="last_checkpoint.pth"):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    checkpoint_filename = os.path.join(checkpoint_dir, filename)
    torch.save(checkpoint, checkpoint_filename)
    print(f"Checkpoint saved at epoch {epoch} as {checkpoint_filename}.")
    print("\n=========================================================================")
    

# Function to load model checkpoints
def load_checkpoint(filename, model, optimizer=None):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    print(f"Checkpoint loaded from epoch {epoch}.")
    return epoch

# Function to calculate IoU per class
def calculate_iou_per_class(pred, target, num_classes):
    iou_per_class = []
    for cls in range(num_classes):
        pred_inds = pred == cls
        target_inds = target == cls
        intersection = (pred_inds & target_inds).sum().float().item()
        union = (pred_inds | target_inds).sum().float().item()
        if union == 0:
            iou_per_class.append(float('nan'))
        else:
            iou_per_class.append(intersection / union)
    return iou_per_class


def train_model_with_csv_and_checkpoint(model, train_dataloader, val_dataloader, criterion, optimizer, num_classes, num_epochs=25, device='cuda:0', csv_filename="resize_metrics.csv", checkpoint_dir="./resize_checkpoints", checkpoint_file=None):
    os.makedirs(checkpoint_dir, exist_ok=True)
    model.to(device)
    print("모델 장치:", next(model.parameters()).device)

    best_val_miou = -1

    # 체크포인트 파일을 불러오고, 이어서 학습할 시작 에폭 설정
    start_epoch = 0
    if checkpoint_file and os.path.isfile(checkpoint_file):
        start_epoch = load_checkpoint(checkpoint_file, model, optimizer)
        print(f"체크포인트 로드 완료. {start_epoch + 1} 에폭부터 재개합니다.")
    else:
        print("체크포인트가 없거나 파일을 찾을 수 없습니다. 처음부터 학습을 시작합니다.")

    # CSV에 기록할 에폭을 66부터 시작하도록 설정
    csv_epoch = 66

    # 학습 루프: `start_epoch + 1`부터 `start_epoch + num_epochs`까지
    for epoch in range(start_epoch + 1, start_epoch + num_epochs + 1):
        dataloader_index = epoch % len(train_dataloader)
        current_dataloader = train_dataloader[dataloader_index]
        
        print("\n=========================================================================")
        print(f"{epoch}/{start_epoch + num_epochs} epoch start with Train Dataloader {dataloader_index + 1}")

        model.train()
        running_loss = 0.0
        train_iou = [0.0] * num_classes
        total_train_samples = 0

        for images, masks in tqdm(current_dataloader, desc=f"Train Epoch {epoch}, Dataloader {dataloader_index + 1}", leave=False):
            images = images.to(device)
            masks = masks.long().to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()

            outputs = torch.argmax(outputs, dim=1)
            running_loss += loss.item() * images.size(0)
            total_train_samples += images.size(0)

            class_iou = calculate_iou_per_class(outputs, masks, num_classes)
            for j in range(num_classes):
                if not np.isnan(class_iou[j]):
                    train_iou[j] += class_iou[j]

        epoch_loss = running_loss / total_train_samples
        train_iou = [x / len(current_dataloader) for x in train_iou]
        train_miou = np.nanmean(train_iou[1:])

        print(f"{epoch} epoch - Train Loss: {epoch_loss:.4f}, Train mIoU: {train_miou:.4f}")

        # 검증 단계 수행
        model.eval()
        val_running_loss = 0.0
        val_iou = [0.0] * num_classes

        with torch.no_grad():
            for images, masks in val_dataloader:
                images = images.to(device)
                masks = masks.long().to(device)

                outputs = model(images)
                loss = criterion(outputs, masks)
                outputs = torch.argmax(outputs, dim=1)
                val_running_loss += loss.item() * images.size(0)

                class_iou = calculate_iou_per_class(outputs, masks, num_classes)
                for j in range(num_classes):
                    if not np.isnan(class_iou[j]):
                        val_iou[j] += class_iou[j]

        val_loss = val_running_loss / len(val_dataloader.dataset)
        val_iou = [x / len(val_dataloader) for x in val_iou]
        val_miou = np.nanmean(val_iou[1:])

        print(f"{epoch} epoch - Validation Loss: {val_loss:.4f}, Validation mIoU: {val_miou:.4f}")
        

        # 강제로 CSV에 기록할 에폭을 49부터 시작하도록 설정
        save_metrics_to_csv(csv_epoch, epoch_loss, val_loss, train_miou, val_miou, train_iou[1:], val_iou[1:])
        csv_epoch += 1  # CSV 에폭을 증가시켜 다음 에폭에 반영

        # 매 에폭마다 체크포인트 저장
        epoch_checkpoint_filename = f"checkpoint_epoch_{csv_epoch}.pth"
        save_checkpoint(csv_epoch, model, optimizer, checkpoint_dir, filename=epoch_checkpoint_filename)

        # 최적의 검증 성능 기준으로 최상의 체크포인트도 업데이트
        if val_miou > best_val_miou:
            best_val_miou = val_miou
            best_checkpoint_filename = f"best_checkpoint_epoch_{csv_epoch}.pth"
            save_checkpoint(csv_epoch, model, optimizer, checkpoint_dir, filename=best_checkpoint_filename)

    # 마지막 에폭 체크포인트 저장
    last_checkpoint_filename = f"last_checkpoint_epoch_{csv_epoch}.pth"
    save_checkpoint(csv_epoch, model, optimizer, checkpoint_dir, filename=last_checkpoint_filename)
    print("훈련 완료.")

In [45]:
import torch.optim as optim

# CUDA 환경 설정
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

# 모델 및 학습 설정
num_classes = 8  # 실제 학습할 클래스 수 (배경 제외)
learning_rate = 0.00005  
weight_decay = 0.0005
num_epochs = 100


# 모델, 옵티마이저, 손실 함수 설정
model = LightweightDeepLabV3(num_classes=num_classes)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# 손실 함수 및 장치 설정 (가중치 없이)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
criterion = torch.nn.CrossEntropyLoss(ignore_index=0).to(device)

# 체크포인트 파일 지정 
checkpoint_file = "./resize_checkpoints/checkpoint_epoch_49.pth"

# 학습 함수 호출
train_model_with_csv_and_checkpoint(
    model=model,
    train_dataloader=train_dataloaders,
    val_dataloader=val_dataloader,
    criterion=criterion,
    optimizer=optimizer,
    num_classes=num_classes,
    num_epochs=num_epochs,
    device=device,
    csv_filename="./resize_metrics.csv",
    checkpoint_dir="./resize_checkpoints",
    checkpoint_file=checkpoint_file
)


모델 장치: cuda:0


/tmp/ipykernel_2775340/882447347.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename)


Checkpoint loaded from epoch 49.
체크포인트 로드 완료. 50 에폭부터 재개합니다.

50/149 epoch start with Train Dataloader 3


50 epoch - Train Loss: 0.2765, Train mIoU: 0.5619
50 epoch - Validation Loss: 0.2979, Validation mIoU: 0.5714
Metrics saved successfully to resize_metrics.csv.
Checkpoint saved at epoch 67 as ./resize_checkpoints/checkpoint_epoch_67.pth.

Checkpoint saved at epoch 67 as ./resize_checkpoints/best_checkpoint_epoch_67.pth.


51/149 epoch start with Train Dataloader 4


51 epoch - Train Loss: 0.2555, Train mIoU: 0.5840
51 epoch - Validation Loss: 0.2918, Validation mIoU: 0.5739
Metrics saved successfully to resize_metrics.csv.
Checkpoint saved at epoch 68 as ./resize_checkpoints/checkpoint_epoch_68.pth.

Checkpoint saved at epoch 68 as ./resize_checkpoints/best_checkpoint_epoch_68.pth.


52/149 epoch start with Train Dataloader 1


...

62 epoch - Train Loss: 0.2559, Train mIoU: 0.5752
62 epoch - Validation Loss: 0.3346, Validation mIoU: 0.5561
Metrics saved successfully to resize_metrics.csv.
Checkpoint saved at epoch 79 as ./resize_checkpoints/checkpoint_epoch_79.pth.


63/149 epoch start with Train Dataloader 4


63 epoch - Train Loss: 0.2513, Train mIoU: 0.5870
63 epoch - Validation Loss: 0.2989, Validation mIoU: 0.5652
Metrics saved successfully to resize_metrics.csv.
Checkpoint saved at epoch 80 as ./resize_checkpoints/checkpoint_epoch_80.pth.


64/149 epoch start with Train Dataloader 1


64 epoch - Train Loss: 0.2514, Train mIoU: 0.5825
64 epoch - Validation Loss: 0.3375, Validation mIoU: 0.5408
Metrics saved successfully to resize_metrics.csv.
Checkpoint saved at epoch 81 as ./resize_checkpoints/checkpoint_epoch_81.pth.


65/149 epoch start with Train Dataloader 2


65 epoch - Train Loss: 0.2230, Train mIoU: 0.5826
65 epoch - Validation Loss: 0.3384, Validation mIoU: 0.5608
Metrics saved successfully to resize_metrics.csv.
Checkpoint saved at epoch 82 as ./resize_checkpoints/checkpoint_epoch_82.pth.


66/149 epoch start with Train Dataloader 3


KeyboardInterrupt: 